In [2]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Import Data

In [3]:
interactions_df = pd.read_csv("full_interactions_dataset.csv")

In [4]:
df = pd.read_csv("full_dataset.csv")

/Users/melissamunz/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df.head()

,Unnamed: 0,timestamp_x,eventType_x,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,...,text,lang,timestamp_y,eventType_y,personId,sessionId,userAgent,userRegion,userCountry,eventStrength
0,3,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,...,All of this work is still very early. The firs...,en,1459270471,VIEW,2873028073541627603,-3228450896145227905,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,SP,BR,1.0
1,4,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,...,The alarm clock wakes me at 8:00 with stream o...,en,1459539433,VIEW,8414731042150985013,4543899740167763020,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR,1.0
2,5,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,...,We're excited to share the Google Data Center ...,en,1459476271,VIEW,-7267769888748948232,-1350132153769633851,Android - Native Mobile App,MG,BR,1.0
3,6,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,...,We're excited to share the Google Data Center ...,en,1459476266,VIEW,-7267769888748948232,-1350132153769633851,Android - Native Mobile App,MG,BR,1.0
4,7,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,...,We're excited to share the Google Data Center ...,en,1459476139,LIKE,-7267769888748948232,-1350132153769633851,NaN,NaN,NaN,2.0


# Using Surprise

Notice how there is no X_train or y_train in our values here. Our only features here are the ratings of other users and items, so we need to keep everything together. What is happening in the train-test split here is that surprise is randomly selecting certain  𝑟𝑖𝑗  for users  𝑢𝑖  and items  𝑖𝑗 . 80% of the ratings are in the training set and 20% in the test set. Let's investigate trainset and testset further.

In [28]:
pip install surprise

Note: you may need to restart the kernel to use updated packages.


In [ ]:
print(f'Length of the training set is: {len(interactions_train)}')
print(f'Length of the testing set is: {len(interactions_test)}')

In [ ]:
#Indexing by person Id to speed up searches

indexed_interactions_df = interactions_df.set_index('personId')

indexed_interactions_train = interactions_train.set_index('personId')

indexed_interactions_test = interactions_test.set_index('personId')

# Models

https://towardsdatascience.com/building-and-testing-recommender-systems-with-surprise-step-by-step-d4ba702ef80b

In [42]:
from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy
from surprise.reader import Reader
from surprise import Dataset

In [34]:
indexed_interactions_df.head()

,Unnamed: 0,contentId,event_strength
personId,,,
-9223121837663643404,0,-8949113594875411859,1.000000
-9223121837663643404,1,-8377626164558006982,1.000000
-9223121837663643404,2,-8208801367848627943,1.000000
-9223121837663643404,3,-8187220755213888616,1.000000
-9223121837663643404,4,-7423191370472335463,3.169925


In [44]:
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(interactions_df[['personId', 'contentId', 'event_strength']], reader)

# Train test split

In [53]:
from surprise.model_selection import train_test_split

train_set, test_set = train_test_split(data, test_size = 0.2, random_state = 42)

In [55]:
from surprise import BaselineOnly

algo = BaselineOnly(bsl_options=bsl_options)
predictions = algo.fit(train_set).test(test_set)
accuracy.rmse(predictions)

Estimating biases using als...
RMSE: 0.6591


0.659117769866205

# Initial Algorithm Testing

In [49]:
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')    

NameError: name 'SVD' is not defined

# Baseline Algorithm

In [58]:
from surprise.model_selection import cross_validate

print('Using ALS')

bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
algo = BaselineOnly(bsl_options=bsl_options)

cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)


Using ALS
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


{'test_rmse': array([0.6635169 , 0.67324034, 0.67081471]),
 'fit_time': (0.11132383346557617, 0.12220406532287598, 0.12019515037536621),
 'test_time': (0.34804391860961914, 0.4254598617553711, 0.2930428981781006)}